In [6]:
import pymongo
from pymongo import MongoClient
import pprint
from datetime import datetime
import re
from IPython.display import clear_output

# Replace XXXX with your connection URI from the Atlas UI
client = MongoClient('mongodb+srv://analytics:<password>@mflix.n69ze95.mongodb.net/test')

runtime_pat = re.compile(r'([0-9]+) min')

for movie in client.mflix.movies.find({}).limit(100):

    fields_to_set = {}
    fields_to_unset = {}

    for k,v in movie.copy().items():
        if v == "" or v == [""]:
            del movie[k]
            fields_to_unset[k] = ""

    if 'director' in movie:
        fields_to_unset['director'] = ""
        fields_to_set['directors'] = movie['director'].split(", ")
    if 'cast' in movie:
        fields_to_set['cast'] = movie['cast'].split(", ")
    if 'writer' in movie:
        fields_to_unset['writer'] = ""
        fields_to_set['writers'] = movie['writer'].split(", ")
    if 'genre' in movie:
        fields_to_unset['genre'] = ""
        fields_to_set['genres'] = movie['genre'].split(", ")
    if 'language' in movie:
        fields_to_unset['language'] = ""
        fields_to_set['languages'] = movie['language'].split(", ")
    if 'country' in movie:
        fields_to_unset['country'] = ""
        fields_to_set['countries'] = movie['country'].split(", ")
        
    if 'fullplot' in movie:
        fields_to_unset['fullplot'] = ""
        fields_to_set['fullPlot'] = movie['fullplot']
    if 'rating' in movie:
        fields_to_unset['rating'] = ""
        fields_to_set['rated'] = movie['rating']

    imdb = {}
    if 'imdbID' in movie:
        fields_to_unset['imdbID'] = ""
        imdb['id'] = movie['imdbID']
    if 'imdbRating' in movie:
        fields_to_unset['imdbRating'] = ""
        imdb['rating'] = movie['imdbRating']
    if 'imdbVotes' in movie:
        fields_to_unset['imdbVotes'] = ""
        imdb['votes'] = movie['imdbVotes']
    if imdb:
        fields_to_set['imdb'] = imdb
        
    if 'released' in movie:
        fields_to_set['released'] = datetime.strptime(movie['released'],
                                                      "%Y-%m-%d")
    if 'lastUpdated' in movie:
        fields_to_set['lastUpdated'] = datetime.strptime(movie['lastUpdated'][0:19],
                                                         "%Y-%m-%d %H:%M:%S")

    if 'runtime' in movie:
        m = runtime_pat.match(movie['runtime']) 
        if m:
            fields_to_set['runtime'] = int(m.group(1))

    update_doc = {}
    if fields_to_set:
        update_doc['$set'] = fields_to_set
    if fields_to_unset:
        update_doc['$unset'] = fields_to_unset
    pprint.pprint(update_doc)

    client.mflix.movies.update_one({'_id': movie['_id']}, update_doc)


{'$set': {'cast': ['Charles Kayser', 'John Ott'],
          'countries': ['USA'],
          'directors': ['William K.L. Dickson'],
          'fullPlot': 'A stationary camera looks at a large anvil with a '
                      'blacksmith behind it and one on either side. The smith '
                      'in the middle draws a heated metal rod from the fire, '
                      'places it on the anvil, and all three begin a rhythmic '
                      'hammering. After several blows, the metal goes back in '
                      'the fire. One smith pulls out a bottle of beer, and '
                      'they each take a swig. Then, out comes the glowing '
                      'metal and the hammering resumes.',
          'genres': ['Short'],
          'imdb': {'id': 5, 'rating': 6.2, 'votes': 1189},
          'rated': 'UNRATED',
          'released': datetime.datetime(1893, 5, 9, 0, 0),
          'runtime': 1},
 '$unset': {'country': '',
            'director': '',
     